# Problem 1: Drug Trial

In this problem, you'll analyze some data from a medical drug trial. There are three exercises worth a total of ten points.

Two of the exercises allow you to use **either** Pandas **or** SQL to solve it. Choose the method that feels is more natural to you.

> **Note**: Because Vocareum only supports the `sqlite3` module with Python 3.5, this problem is tied to that version (rather than 3.6). As such, some care may be needed to ensure that you are only using features available in the corresponding versions of Python and supporting libraries (e.g., `pandas`, `sqlite3`).

## Setup

Run the following few code cells, which will load the modules and sample data you'll need for this problem.

In [1]:
import sys
import pandas as pd
import numpy as np
import sqlite3 as db

print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("sqlite3 version: {}".format(db.version))

from IPython.display import display
from cse6040utils import canonicalize_tibble, tibbles_are_equivalent

Python version: 3.6.5 | packaged by conda-forge | (default, Apr  6 2018, 13:39:56) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
pandas version: 0.23.4
numpy version: 1.13.3
sqlite3 version: 2.6.0


In [2]:
from cse6040utils import download_all

datasets = {'Drugs_soln.csv': '6df060bde8dea48986dc12650a4fbef5',
            'avg_dose_soln.csv': 'f604e3da488d489792fec220ada738f8',
            'drugs.csv': '33bb1fa5068069a483a6e05fafde40d0',
            'nst_count_soln.csv': '7519ad4764eb238a9120fa7cd1f006de',
            'nst_count_soln--corrected.csv': '81f801cd20775a51f92a1b28593c0915',
            'swt_count_soln.csv': 'fbbb7368d31856665c3e5e1b19d93d65'}

DATA_SUFFIX = "drug-trials/"
data_paths = download_all(datasets, local_suffix=DATA_SUFFIX, url_suffix=DATA_SUFFIX)
    
print("\n(All data appears to be ready.)")

'Drugs_soln.csv' is ready!
'avg_dose_soln.csv' is ready!
'drugs.csv' is ready!
'nst_count_soln.csv' is ready!
'nst_count_soln--corrected.csv' is ready!
'swt_count_soln.csv' is ready!

(All data appears to be ready.)


## The data

Company XYZ currently uses Medication A to treat all its patients and is considering a switch to Medication B. A critical part of the evaluation of Medication B is how much of it would be used among XYZ’s patients.

The company did a trial of Medication B. The data in the accompanying CSV file, `Drugs.csv`, is data taken from roughly 130 patients at least 2 months before switching medications and up to 3 months while on the new medication.

A patient can be taking medication A or medication B, but cannot be taking both at the same time.

The following code cell will read this data and store it in a dataframe named `Drugs`.

In [3]:
Drugs = pd.read_csv(data_paths['drugs.csv'], header=0)
assert len(Drugs) == 2022
Drugs.head()

ID    Med Admin Date     Units
0   1  Med A     7/2/12  1,500.00
1   1  Med A     7/6/12  1,500.00
2   1  Med A     7/9/12  1,500.00
3   1  Med A    7/11/12  1,500.00
4   1  Med A    7/13/12  1,500.00

In [4]:
Drugs.shape


(2022, 4)

In [5]:
Drugs.ID.nunique()

129

Each row indicates that a patient (identified by his or her `'ID'`) took one **dose** of a particular drug on a particular day. The size of the dose was `'Units'`.

## Exercises

**Exercise 0** (1 points). All you have to do is read the code in the following code cell and run it. You should observe the following.

First, the `'Med'`, `'Admin Date'`, and `'Units'` columns are stored as strings initially.

Secondly, there are some handy functions in Pandas to change the `'Admin Date'` and '`Units`' columns into more "natural" native Python types, namely, a floating-point type and a Python `datetime` type, respectively. Indeed, once in this form, it is easy to use Pandas to, say, extract the month as its own column.

In [6]:
# Observe types:
for col in ['Med', 'Admin Date', 'Units']:
    print("Column '{}' has type {}.".format(col, type(Drugs[col].iloc[0])))
    
# Convert strings to "natural" types:
Drugs = pd.read_csv(data_paths['drugs.csv'], header=0)
Drugs['Units'] = pd.to_numeric(Drugs['Units'].str.replace(',',''), errors='coerce')
Drugs['Admin Date'] = pd.to_datetime(Drugs['Admin Date'], infer_datetime_format=True)
Drugs['Month'] = Drugs['Admin Date'].dt.month

print ("\nFive random records from the `Drugs` table:")
display(Drugs.iloc[np.random.choice (len (Drugs), 5)])

assert Drugs['Units'].dtype == 'float64'
assert Drugs['Month'].dtype == 'int64'

Column 'Med' has type <class 'str'>.
Column 'Admin Date' has type <class 'str'>.
Column 'Units' has type <class 'str'>.

Five random records from the `Drugs` table:


ID    Med Admin Date   Units  Month
454    29  Med A 2012-07-21  4400.0      7
1928  124  Med A 2012-07-31  6000.0      7
1928  124  Med A 2012-07-31  6000.0      7
37      2  Med A 2012-07-19  1300.0      7
1777  114  Med A 2012-07-26  3700.0      7

**Exercise 1** (1 point). Again, all you need to do is read and run the following code cell. It creates an SQLite database file named `drug_trial.db` and copies the Pandas dataframe from above into it as a table named `Drugs`.

The `conn` variable holds a live connection to this data.

In [7]:
# Import Drugs_soln dataframe above to sqlite database
# Connect to a database (or create one if it doesn't exist)
conn = db.connect('drug_trial.db')
Drugs.to_sql('Drugs', conn, if_exists='replace', index=False)
pd.read_sql_query('SELECT * FROM Drugs LIMIT 5', conn)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


ID    Med           Admin Date   Units  Month
0   1  Med A  2012-07-02 00:00:00  1500.0      7
1   1  Med A  2012-07-06 00:00:00  1500.0      7
2   1  Med A  2012-07-09 00:00:00  1500.0      7
3   1  Med A  2012-07-11 00:00:00  1500.0      7
4   1  Med A  2012-07-13 00:00:00  1500.0      7

**Exercise 2** (2 points). **Suppose you want to know the average dose, for each medication (A and B) and month ranging from July to November.**

For example, it will turn out that in July the average dose of drug A was 5,129.56 units (rounded to two decimal places), and in September the average dose of drug B was 7.04.

Write some code to perform this calculation. Store your results in a Pandas data frame named `avg_dose` having the following three columns:
- `'Month'`: The month;
- `'Med'`: The medication, either `'Med A'` and `'Med B'`;
- `'Units'`: The average dose, **rounded to 2 decimal digits**.

> You can write either Pandas code or SQL code. If using Pandas, the data exists in the `Drugs` dataframe; if using SQL, the `conn` database connection holds a table named `Drugs`.

In [8]:
Drugs.columns

Index(['ID', 'Med', 'Admin Date', 'Units', 'Month'], dtype='object')

In [9]:
Drugs.groupby(['Month', 'Med']).mean().reset_index()

Month    Med          ID         Units
0      7  Med A   66.442193   5129.558999
1      8  Med A   67.026482   5645.775536
2      9  Med A   68.906250   5311.875000
3      9  Med B   66.947368      7.039474
4     10  Med B   64.720588      5.779412
5     11  Med A   35.142857  10757.142857
6     11  Med B   67.946667      5.600000
7     12  Med A   47.666667   3400.000000
8     12  Med B  108.000000      1.000000

In [10]:
#
avg_dose = Drugs.groupby(['Month', 'Med']).mean()
avg_dose['Units'] = [round(unit, 2) for unit in avg_dose['Units']]
avg_dose.drop('ID', axis=1, inplace=True)
avg_dose.reset_index(drop=False, inplace=True)
avg_dose = avg_dose[avg_dose['Month'] != 12]
#

# Show your solution:
display(avg_dose)

Month    Med     Units
0      7  Med A   5129.56
1      8  Med A   5645.78
2      9  Med A   5311.88
3      9  Med B      7.04
4     10  Med B      5.78
5     11  Med A  10757.14
6     11  Med B      5.60

In [11]:
# Test code
# Read what we believe is the exact result (up to permutations)
avg_dose_soln = pd.read_csv(data_paths['avg_dose_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'avg_dose' in globals(), "You need to store your results in a dataframe named `avg_dose`."
assert type(avg_dose) is type(pd.DataFrame()), "`avg_dose` does not appear to be a Pandas dataframe."
assert len(avg_dose) == len(avg_dose_soln), "The number of rows of `avg_dose` does not match our solution."
assert set(avg_dose.columns) == set(['Month', 'Med', 'Units']), "Your table does not have the right set of columns."

assert tibbles_are_equivalent(avg_dose, avg_dose_soln)
print("\n(Passed!)")


(Passed!)


**Exercise 3** (6 points). For each month, write some code to calculate the following:
- (3 points) How many patients switched from medication A to medication B? Store youre results in a Pandas dataframe named `swt_count`.
- (3 points) How many patients started on medication B, having never been on medication A before? Store your results in a Pandas dataframe named `nst_count`.

The two dataframes should have two columns: `Month` and `Count`. Again, you can choose to use SQL queries or Pandas directly to generate these dataframes.

> If it's helpful, recall that patients can only be switched from medication A to medication B, but not from B back to A.

In [12]:
all_patients = Drugs.groupby(['ID', 'Med', 'Month']).mean().reset_index()
all_patients.head(10)

ID    Med  Month        Units
0   1  Med A      7  1554.545455
1   1  Med A      8  1761.538462
2   1  Med A      9  1900.000000
3   1  Med B      9     4.000000
4   1  Med B     10     3.000000
5   1  Med B     11     1.000000
6   2  Med A      7  1271.428571
7   2  Med A      8  1300.000000
8   2  Med B      9     3.000000
9   3  Med A      7   100.000000

In [13]:
from collections import defaultdict

patient_month_switched_d = defaultdict(int)
patient_noswitch_d = defaultdict(int)
# Patient = key
# Month = count of patient that switched from Med A to Med B

for patient in all_patients['ID'].unique().tolist():
    patient_month_switched_d[patient] = 0
    patient_noswitch_d[patient] = 0

# Now, go through the dataframe patient by patient and see if they switched
# if the patient switched drug (assuming the month we increment is the month they switched)
for i in range(len(all_patients)):
    patient = all_patients.loc[i, 'ID']

    sub_df = all_patients[all_patients['ID'] == patient]
    sub_df.reset_index(drop=True, inplace=True)
    
    # this means the patient did not switch drugs (only 1 unique drug to the patient)
    if sub_df['Med'].nunique() == 1:
        patient_month_switched_d.pop(patient, None)
        
        # for the case where the patient has only been on Med B
        if 'Med B' in sub_df['Med'].unique().tolist():
            # find the index where the patient began Med B
            patient_noswitch_d[patient] = sub_df.loc[0, 'Month']
            
        # to handle cases where the patient was only on Med A and never switched
        else:
            pass
        
    # this means the patient switched drugs
    else:
        # pop the patient out of the patient_noswitch_d
        patient_noswitch_d.pop(patient, None)
        
        # grab the 1st index in the np.where statement becuase that'll be the one where the patient switched
        medb_idx = np.where(sub_df['Med'] == 'Med B')[0]
        month_patient_switched = sub_df.loc[medb_idx.tolist()[0], 'Month']
        
        # attach that month to the dictionary
        patient_month_switched_d[patient] = month_patient_switched
    
patient_month_switched_d = dict(patient_month_switched_d)
patient_noswitch_d = dict(patient_noswitch_d)

In [14]:
# Write your solution to compute `swt_count` in this code cell.

#
my_df = pd.DataFrame.from_dict(patient_month_switched_d, orient='index').reset_index()
my_df.rename(columns={'index':'patient',
                      0: 'Month'}, inplace=True)

swt_count = my_df.groupby('Month').count()
swt_count.reset_index(drop=False, inplace=True)
swt_count.rename(columns={'patient':'Count'}, inplace=True)
swt_count
#

Month  Count
0      9     71
1     10     10

In [15]:
# Test code for exercise_a
# Read what we believe is the exact result
swt_count_soln = pd.read_csv(data_paths['swt_count_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'swt_count' in globals ()
assert type (swt_count) is type (pd.DataFrame ())
assert len (swt_count) == len (swt_count_soln)
assert set (swt_count.columns) == set (['Month', 'Count'])

print ("Number of patients who switched from Med A to Med B each month:")
display (swt_count)

assert tibbles_are_equivalent (swt_count, swt_count_soln)
print ("\n(Passed!)")

Number of patients who switched from Med A to Med B each month:


Month  Count
0      9     71
1     10     10


(Passed!)


In [16]:
# Write your solution to compute `nst_count` in this code cell.

#
my_df = pd.DataFrame.from_dict(patient_noswitch_d, orient='index').reset_index()
my_df = my_df[my_df[0] != 0]
my_df.rename(columns={'index': 'patient', 0:'Month'}, inplace=True)
my_df.reset_index(drop=True, inplace=True)

nst_count = my_df.groupby('Month').count()
nst_count.reset_index(inplace=True)
nst_count.rename(columns={'patient':'Count'}, inplace=True)
nst_count
#


Month  Count
0      9      5
1     10      5
2     11      6

In [17]:
# Test code for exercise_b
# Read what we believe is the exact result
nst_count_soln_corrected = pd.read_csv(data_paths['nst_count_soln--corrected.csv'])
nst_count_soln_ok = pd.read_csv(data_paths['nst_count_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'nst_count' in globals ()
assert type (nst_count) is type (pd.DataFrame ())
assert (len (nst_count) == len (nst_count_soln_corrected)) or (len (nst_count) == len (nst_count_soln_ok))
assert set (nst_count.columns) == set (['Month', 'Count'])

print ("Number of patients who newly start Med B each month:")
display (nst_count)

assert tibbles_are_equivalent(nst_count, nst_count_soln_ok) \
       or tibbles_are_equivalent(nst_count, nst_count_soln_corrected)
print ("\n(Passed!)")

Number of patients who newly start Med B each month:


Month  Count
0      9      5
1     10      5
2     11      6


(Passed!)


In [18]:
# Some cleanup code
conn.close()

**Fin!** Well done! If you have successfully completed this problem, move on to the next one. Good luck!